# Business Problem : Lending Club Case Study - EDA

In [ ]:
# Importing all Depenedent libraries.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

In [ ]:
# As per initial step, we are looking into the provided loan input file to check no.of rows and their data types

loan_input = pd.read_csv("C:\Python_Training\Lendingclubcasestudy\loan\loan.csv",dtype=object)
#loan_input = loan_input[loan_input['loan_status'] != 'Current']

# Verifying shape & count of non-null values
loan_input.shape
loan_input.info(verbose=True,show_counts=True)

In [ ]:
data_cols = loan_input.describe().columns
data_cols

In [ ]:
continuous = [col for col in loan_input.columns if col in data_cols]
continuous

In [ ]:
categorical = [col for col in loan_input.columns if col not in data_cols]
categorical

In [ ]:
# Data cleansing: 
# As per above Drop columns where we have mostly null values in given below range criteria and check dimensions of data
loan_input.drop(loan_input.iloc[:, 53:78], inplace = True, axis = 1) 
loan_input.shape
loan_input.info(verbose=True,show_counts=True)

In [ ]:
# Data cleansing: 
# As per above Drop columns where we have mostly null values in given below range criteria and check dimensions of data
loan_input.drop(loan_input.iloc[:, 55:80], inplace = True, axis = 1) 
loan_input.shape
loan_input.info(verbose=True,show_counts=True)

In [ ]:
# Data cleansing: 
# Dropping extra columns having null values and check input dimensions

drop_columns_list1 = ["mths_since_last_major_derog","tot_hi_cred_lim","total_bal_ex_mort","total_bc_limit","total_il_high_credit_limit"]
loan_input.drop(labels = drop_columns_list1, axis =1, inplace=True)

drop_columns_list2 = ["member_id","url","emp_title","zip_code","tax_liens"]
loan_input.drop(labels = drop_columns_list2, axis =1, inplace=True)


loan_input.shape

In [ ]:
loan_input.info(verbose=True,show_counts=True)

In [ ]:
# Data cleansing: 
# Check for duplicate values if any

duplicate_values=loan_input[loan_input.duplicated()]
duplicate_values

In [ ]:
loan_input['loan_amnt'].describe()

#Observation: Numerical data are not yielding expected statistical info - we need to consider their data type conversion.

In [ ]:
# Data cleansing: 
# As part of data type fix, Stripping special character % from intrest rate & revol_util column to be utilized in further analysis.
loan_input['int_rate'] = loan_input['int_rate'].str.strip('%')
loan_input['revol_util'] = loan_input['revol_util'].str.strip('%')

# As part of data type fix, ensuring employment length to have only numerics
loan_input.emp_length.fillna('0',inplace=True)
loan_input['emp_length']=loan_input.emp_length.str.extract('(\d+)')

# Remove the word 'months' from the column 'term',typecast it to 'int' and save in a new column 'term_in_months'
loan_input['term_in_months']=loan_input['term'].apply(lambda x: x.split()[0]).astype('int')

In [ ]:
loan_input['emp_length'].value_counts(dropna=False)

In [ ]:
# # Data cleansing: 
# Filling pub_rec_bankruptcies for further analysis.
print("Before cleaning:")
print(loan_input.pub_rec_bankruptcies.isnull().sum())
print("After cleaning:")
loan_input.pub_rec_bankruptcies.fillna('Not Known',inplace=True)
print(loan_input.pub_rec_bankruptcies.isnull().sum())

In [ ]:
data_cols = loan_input.describe().columns
data_cols

In [ ]:
# Data cleansing: 
# As part of fixing Data, handling type conversion for numerical columns to derive more insights

cols_tonumeric = ['loan_amnt','funded_amnt','int_rate','funded_amnt_inv','installment','annual_inc','dti','emp_length','total_pymnt','recoveries','term_in_months']
loan_input[cols_tonumeric] = loan_input[cols_tonumeric].apply(pd.to_numeric)
pd.options.display.float_format = '{:.2f}'.format 

In [ ]:
loan_input['loan_amnt'].describe()
#Observation: Numerical data are yielding expected statistical info.

In [ ]:
loan_input['annual_inc'].describe()
#Observation: Numerical data are yielding expected statistical info.

In [ ]:
loan_input['loan_status']

In [ ]:
loan_input['loan_status'].value_counts()

In [ ]:
# Data cleansing: 
# Exclude current loans for further evaluation consider only "Fully paid " and "Charged off".

loan_input=loan_input[loan_input['loan_status'].isin(['Fully Paid','Charged Off'])]

In [ ]:
# Remove the word 'months' from the column 'term',typecast it to 'int' and save in a new column 'term_in_months'
loan_input['term_in_months']=loan_input['term'].apply(lambda x: x.split()[0]).astype('int')  #float

In [ ]:
# Delete the 'term' column as it is no more required
loan_input = loan_input.drop(['term'], axis=1)

## Derived column analysis for further analysis from existing columns

In [ ]:
# Checking for Derived columns from issue_d and creating month and year columns separately for further anaysis

loan_input.issue_d = pd.to_datetime(loan_input.issue_d, format='%b-%y')
loan_input['year'] = loan_input['issue_d'].dt.year
loan_input ['month'] = loan_input['issue_d'].dt.month

# Drop further columns and check dimensions of Data frame
drop_columns_list3 = ["issue_d","desc","mths_since_last_delinq","mths_since_last_record","next_pymnt_d","collections_12_mths_ex_med","chargeoff_within_12_mths"]
loan_input.drop(labels = drop_columns_list3, axis =1, inplace=True)


loan_input.shape

In [ ]:
loan_input['year'].value_counts()

In [ ]:
# Create a column 'loan_defaulted' which will have value 1 if loan was defaulted else 0 if not defaulted
loan_input['loan_defaulted']=loan_input['loan_status'].apply(lambda x: 1 if x=='Charged Off' else 0 )

## Univariate Analysis process:

In [ ]:
# Annual_inc: Checking for outliers in annual_inc column - to derive statistical info of data spread 
# as it's one of the driving eligibility criteria

loan_input['annual_inc'].describe()

In [ ]:
sns.boxplot(loan_input['annual_inc'])
plt.title('Annual income Data spread Before removal of outliers')
plt.xlabel("Borrowers Annual Income")
plt.show()

In [ ]:
sns.boxplot(loan_input['annual_inc'])
plt.xlabel("Borrowers Annual Income")
plt.show()

In [ ]:
# Check for existence of data skew - indicates existence of outliers
loan_input['annual_inc'].skew()

In [ ]:
percentile25 = loan_input['annual_inc'].quantile(0.1)
percentile95 = loan_input['annual_inc'].quantile(0.99)

IQR = percentile95-percentile25

In [ ]:
upper_limit = percentile95 + 1.5 * IQR
lower_limit = percentile25 - 1.5 * IQR

In [ ]:
#loan_input = loan_input[(loan_input['annual_inc'] < upper_limit) & (loan_input['annual_inc'] > lower_limit)]
loan_input = loan_input[loan_input['annual_inc'] <= percentile95]
loan_input.shape


In [ ]:
# Check for existence of data skew - after removal of outliers it got improved
loan_input['annual_inc'].skew()

In [ ]:
# Checking annual income details after removal of outliers

plt.title('Annual income Data spread After removal of outliers')
sns.boxplot(loan_input['annual_inc'])
plt.show()

### Observations:

In [ ]:
# Analysis on Borrower's year of employment

In [ ]:
sns.countplot(x=loan_input["emp_length"], data=loan_input,palette='PuBu')
plt.xlabel('Employee Working Experience in Years')
plt.show()

### Observations: 
 #### Most of the employees are of 1 years and 10 years of work experience

In [ ]:
# Analysis on Borrower's Home owner ship
loan_input.groupby('home_ownership').home_ownership.count()

In [ ]:
sns.countplot(x='home_ownership', data=loan_input,order=loan_input.home_ownership.value_counts().index,palette='BuPu')
plt.xlabel('Home Ownership')
plt.show()

### Observations: 
 #### Most of borrowers have houses on Rent and on Mortgage. 
 #### Very few have ownership house

In [ ]:
# Analysis on Loan status:
#sns.countplot(x= loan_input["loan_status"].sort_values())
sns.countplot(x=loan_input["loan_status"].sort_values(),data=loan_input,hue='loan_status',palette='coolwarm')
plt.show()

In [ ]:
loan_input['loan_status'].value_counts()

In [ ]:
# Determine the percentage of borrower who paid and defaulted
(loan_input.loan_status.value_counts()*100)/len(loan_input)

In [ ]:
loan_input["loan_status"].value_counts(1).plot.pie(autopct='%1.1f%%')
plt.ylabel("")
plt.xlabel('')
plt.title("Loan Status Analysis")
plt.show()

### Observations:
   #### Data related to loan status "fully paid" > "charged off".
####  Defaulters: 14.59% 
#### Fully paid: 85.42%

In [ ]:
# Loan amt: 
# Deriving statistical info of data spread 
loan_input['loan_amnt'].describe()

In [ ]:
# Check distribution of three loan amount fields.
sns.set(style="whitegrid")
plt.figure(figsize=(30, 15),facecolor='w')
plt.subplots_adjust(left=0.1,
                    bottom=0.1,
                    right=0.9,
                    top=0.9,
                    wspace=0.4,
                    hspace=0.4)
plt.subplot(2, 3, 1)
ax1= sns.histplot(loan_input['loan_amnt'], bins = 30, kde=True)
ax1.set_title("Loan amount distribution Plot", fontsize=15, color='b')
ax1.set_xlabel('Loan amount',fontsize=14,color='b')
plt.subplot(2, 3, 2)
plt.title("Funded amount distribution Plot", fontsize=14, color='b')
sns.histplot(loan_input['funded_amnt'], bins = 30, kde=True)
plt.subplot(2, 3, 3)
plt.title("Funded amount Inv. - distribution Plot", fontsize=14, color='b')
sns.histplot(loan_input['funded_amnt_inv'], bins = 30, kde=True)
plt.show()

In [ ]:
sns.boxplot( y=loan_input["loan_amnt"] )
plt.ylabel('Loan Amount')
plt.title("Loan Amount variation")
plt.show()

### Observations:
   #### Loan amount varies from 5000 USD to 35000 USD 
   #### Most of the loan(s) were taken between the amount of 5000 USD to 15000 USD

In [ ]:
# Check distribution of Interest rate.
plt.figure(figsize=(20,10),facecolor='w')
plt.subplots_adjust(left=0.1,
                    bottom=0.1,
                    right=0.9,
                    top=0.9,
                    wspace=0.4,
                    hspace=0.4)
sns.set(style="whitegrid")
# subplot 1
plt.subplot(2, 4, 1)
ax1 = sns.histplot(loan_input['int_rate'],bins = 30, kde=True)
ax1.set_title("Interest Rate distribution", fontsize=14, color='b')
ax1.set_xlabel('Interest Rate',fontsize=14,color='b')
# subplot 2
plt.subplot(2, 4, 2)
ax2 = sns.boxplot(y=loan_input['int_rate'],palette="Blues")
ax2.set_title("Interest Rate Boxplot", fontsize=14, color='b')
ax2.set_ylabel('Interest Rate',fontsize=14,color='b')
plt.show()

### Observations:

   #### Most of the loan(s) are in range of 10 to 15 % Interest Rates

In [ ]:
plt.figure(figsize=(20,10),facecolor='w')
plt.subplot(1,2,1)
ax1 = sns.histplot(data=loan_input["dti"])
ax1.set_xlabel('dti',fontsize=18,color='b')
plt.subplot(1,2,2)
ax2=sns.boxplot(data=loan_input["dti"],palette="Blues")
plt.suptitle('Debt to Income Analysis',fontsize=20, color='b')
plt.show()

### Observations:
#### DTI < 30 for all loans and median lies between 10-15.

## Segmented Univariate Analysis:

In [ ]:
# Check the loan status against home ownership type:
sns.set(style="whitegrid")
ax = sns.countplot(x='home_ownership',hue='loan_status', data=loan_input, palette='coolwarm')
ax.set_title('Home ownership Vs Loan status',fontsize=14,color = 'b')
ax.set_xlabel('Home ownership',fontsize=14,color='b')
plt.xticks(rotation=90)
plt.show()

### Observations:
#### Most Customers with home ownership type as Rent and Mortgage are the borrowers who are at having high default rate

In [ ]:
# Check the loan status against purpose of Loan:
sns.set(style="whitegrid")
plt.figure(figsize=(10, 5))
ax = sns.countplot(x='purpose', hue="loan_status", data=loan_input, palette ='bwr')
ax.set_title('Loan status vs Purpose for which loans granted',fontsize=14,color = 'b')
ax.set_ylabel('Chargedoff Proportion',fontsize=14,color = 'b')
ax.set_xlabel('Purpose',fontsize=14,color='b')
plt.xticks(rotation=90)
plt.show()

### Observations:
#### Highest number of defaulters are mostly the borrower who took loan for Debt Consolidation purpose

In [ ]:
# Check the loan status against Term of Loan:
sns.set(style="whitegrid")
sns.countplot(x='term_in_months', hue="loan_status", data=loan_input, palette ='coolwarm')
plt.title('Loan status vs Loan payment term in months',fontsize=14,color = 'b')
plt.xticks(rotation=90)
plt.show()

###  Observations :
#### Most of borrowers who had taken loan to repay in 60months had more % of number of applicants getting 
#### charged off as compared to applicants who had taken loan for 36 months.

## Bivariate Analysis:

In [ ]:
# Derived columns
# categorise loan amounts into buckets which will help in analysis further in bivariate analysis.
loan_input['loan_amnt_cats'] = pd.cut(loan_input['loan_amnt'], [0, 7000, 14000, 21000, 28000, 35000], labels=['0-7000', '7000-14000', '14000-21000', '21000-28000', '28000 +'])

# categorise annual incomes into buckets which will help in analysis further in bivariate analysis.
loan_input['annual_inc_cats'] = pd.cut(loan_input['annual_inc'], [0, 20000, 40000, 60000, 80000,1000000], labels=['0-20000', '20000-40000', '40000-60000', '60000-80000', '80000 +'])

# categorise intrest rates into buckets which will help in analysis further in bivariate analysis.
loan_input['int_rate_cats'] = pd.cut(loan_input['int_rate'], [0, 10, 12.5, 16, 20], labels=['0-10', '10-13', '12.5-16', '16 +'])

# categorise dti into buckets for bivariate analysis.
loan_input['dti_cats'] = pd.cut(loan_input['dti'], [0, 5, 10, 15, 20, 25], labels=['0-5', '05-10', '10-15', '15-20', '25+'])

type(loan_input['loan_amnt_cats'])

In [ ]:
loan_input.shape

In [ ]:
#loan_input['loan_status'] = loan_input['loan_status'].astype('category').cat.codes

In [ ]:
#Bivariate Analysis on annual income against Chargedoff_Proportion -

inc_range_vs_loan = loan_input.groupby(['annual_inc_cats', 'loan_status']).loan_status.count().unstack().fillna(0).reset_index()
inc_range_vs_loan['Total'] = inc_range_vs_loan['Charged Off'] + inc_range_vs_loan['Fully Paid'] 
inc_range_vs_loan['Chargedoff_Proportion'] = inc_range_vs_loan['Charged Off'] / inc_range_vs_loan['Total']
inc_range_vs_loan.sort_values('Chargedoff_Proportion', ascending=False)

In [ ]:
# Loan Defaulter(s) analysis with respect to Annual income and Loan status
# Create a barplot with Annual Income against Laon defaulted
plt.figure(figsize=(10, 5))
sns.set(style="whitegrid")
rates = pd.qcut(loan_input['annual_inc'], 10)
df = pd.DataFrame({ 'annual_inc' : rates, 'loan_defaulted' : loan_input['loan_defaulted']})

fig, ax1 = plt.subplots(figsize=(10, 5),facecolor='w')
ax1.set_title('Annual Income vs Loan status',fontsize=15,color = 'b')
ax1=sns.barplot(y= 'Chargedoff_Proportion', x='annual_inc_cats', data=inc_range_vs_loan, palette ='coolwarm', errorbar=None)
ax1.set_ylabel('Chargedoff Proportion',fontsize=14,color = 'b')
ax1.set_xlabel('Annual Income',fontsize=14,color='b')
plt.xticks(rotation=90)
plt.show()

In [ ]:
#loan_input['grade']=loan_input['grade'].astype('category').cat.codes

In [ ]:
#loan_corr_input = loan_input.select_dtypes(exclude=['string','object'])
#loan_corr_input.info()

In [ ]:
# Evaluating correlation:

#loan_input_corr = loan_corr_input.corr()
#type(loan_input_corr)

#data.corr()['loan_status'].drop('loan_status').sort_values()

In [ ]:
#sns.set(font_scale=1.0)
#sns.heatmap(loan_input_corr,annot=True,cmap="Greens")
#plt.show()

In [ ]:
#sns.set(font_scale=1.0)
#dataplot = sns.heatmap(loan_input_corr, cmap="Greens", annot=True, fmt='.2f')

#sns.clustermap(loan_input_corr,annot=True,figsize=(12, 8),cmap="Greens", fmt='.2f')
  
# displaying correlation view 
#plt.figure(figsize=[50,30])
#plt.show()


In [ ]:
#loan_input_corr.style.format(precision=2)
#loan_input_corr.style.background_gradient(cmap='Greens').set_precision

In [ ]:
# Loan Defaulter(s) analysis with respect to year of loan granted and verification status.
# Plot a bar plot with loan_defaulted against loan_funded_year
sns.set(style="whitegrid")
plt.figure(figsize=(10, 5))
sns.barplot(data=loan_input, y='loan_defaulted', x='year',hue='verification_status', palette ='coolwarm', errorbar=None)
plt.title('Loan defaulter(s) count for each Year with verification status',fontsize=15,color = 'b')
plt.xlabel('Loan Funded Year',fontsize=15,color = 'b')
plt.ylabel('Loan Defaulted',fontsize=15,color = 'b')
plt.show()

###  Observations :
#### Increasing number of defaulters over the year 2008.
#### Number of defaulters have increased during 2008 - may be due to Global economy recession.

In [ ]:
# Loan Defaulter(s) analysis with respect to individual state(s) level.
plt.figure(figsize=(15,15))
sns.set(style="whitegrid")
dataset = loan_input[loan_input['loan_status']=='Charged Off'].groupby('addr_state')['loan_status'].count().reset_index()
sns.barplot(y='addr_state', x='loan_status', data=dataset, palette='coolwarm')
plt.title('Loan defaulter(s) count at each State level',fontsize=15,color = 'b')
plt.xlabel('Count of defaulters by State',fontsize=15,color = 'b')
plt.ylabel('State',fontsize=15,color = 'b')
plt.show()

###  Observations :
#### Califorinia (CA) , Florida (FL) and New York (NY) are top 3 states with more Loan defaulters count.

In [ ]:
# Bivariate Analysis on purpose of Loan against Chargedoff_Proportion.

purpose_vs_loan = loan_input.groupby(['purpose', 'loan_status']).loan_status.count().unstack().fillna(0).reset_index()
purpose_vs_loan['Total'] = purpose_vs_loan['Charged Off'] + purpose_vs_loan['Fully Paid'] 
purpose_vs_loan['Chargedoff_Proportion'] = purpose_vs_loan['Charged Off'] / purpose_vs_loan['Total']
purpose_vs_loan.sort_values('Chargedoff_Proportion', ascending=False)

In [ ]:
# Loan defaulter analysis with respect to Purpose of Loans
fig, ax1 = plt.subplots(figsize=(14, 8),facecolor='w')
sns.set(style="whitegrid")
ax1.set_title('Purpose of Loans vs Chargedoff Proportion',fontsize=15,color = 'b')
ax1=sns.barplot(y='purpose', x='Chargedoff_Proportion', data=purpose_vs_loan, palette='coolwarm')
ax1.set_ylabel('Purpose of Loans',fontsize=14,color='b')
ax1.set_xlabel('Chargedoff Proportion',fontsize=14,color = 'b')
plt.show()

In [ ]:
# Puprose of loan vs Loan amount analysis at Box plot level

plt.figure(figsize=(12,8),facecolor='w')
ax = sns.barplot(y='purpose', x='loan_amnt', data =loan_input,palette='coolwarm')
ax.set_title('Purpose of Loan vs Loan Amount',fontsize=15,color='b')
ax.set_ylabel('Purpose of Loan',fontsize=14,color = 'b')
ax.set_xlabel('Loan Amount',fontsize=14,color = 'b')
plt.show()

###  Observations :
#### Small Business applicants have high chances of getting charged off as their revenue generation may be low.

In [ ]:
# Borrower's Grade Vs Chargedoff_Proportion analysis:
grade_vs_loan = loan_input.groupby(['grade', 'loan_status']).loan_status.count().unstack().fillna(0).reset_index()
grade_vs_loan['Total'] = grade_vs_loan['Charged Off'] + grade_vs_loan['Fully Paid'] 
grade_vs_loan['Chargedoff_Proportion'] = grade_vs_loan['Charged Off'] / grade_vs_loan['Total']
grade_vs_loan.sort_values('Chargedoff_Proportion', ascending=False)


In [ ]:
Loan_Temp = pd.pivot_table(data=loan_input, index=loan_input["grade"], columns=loan_input["loan_status"],values="loan_status", aggfunc='count')
Loan_Temp.plot(kind="bar", stacked="True", figsize=[8,6])
plt.ylabel("Count of Loan Status")
plt.xlabel("Grade")
plt.title("Grade Vs Loan Status") 
plt.show()

In [ ]:
# Borrower's Grade Vs Chargedoff_Proportion analysis:
fig, ax1 = plt.subplots(figsize=(14, 8),facecolor='w')
ax1.set_title('Grades vs Chargedoff Proportion',fontsize=15,color='b')
ax1=sns.barplot(x='grade', y='Chargedoff_Proportion', data=grade_vs_loan, palette='coolwarm')
ax1.set_xlabel('Grades',fontsize=14,color='b')
ax1.set_ylabel('Chargedoff Proportion',fontsize=14,color ='b')
plt.show()

### Observations:
#### Grade "A" has very less chances of charged off.
#### Grade "F" and "G" have very high chances of charged off.
#### Chances of charged off is increasing with grade moving from "A" towards "G"

In [ ]:
# Annual income Vs Grade for every loan status.
plt.figure(figsize=(10,8), dpi=80, facecolor='w', edgecolor='k')
ax=sns.barplot(x='grade', y='annual_inc', hue="loan_status", data=loan_input, palette='coolwarm', estimator=np.mean,order='ABCDEFG')
ax.set_title('Grades vs Annual income',fontsize=15,color='b')
ax.set_ylabel('Annual income',fontsize=14,color = 'b')
ax.set_xlabel('Grades',fontsize=14,color = 'b')
plt.show()

### Observations:
#### 'Charged off' is happening at Grade F than compared to other Grades

In [ ]:
# sub grades Vs Chargedoff_Proportion

grade_vs_loan = loan_input.groupby(['sub_grade', 'loan_status']).loan_status.count().unstack().fillna(0).reset_index()
grade_vs_loan['Total'] = grade_vs_loan['Charged Off'] + grade_vs_loan['Fully Paid'] 
grade_vs_loan['Chargedoff_Proportion'] = grade_vs_loan['Charged Off'] / grade_vs_loan['Total']
grade_vs_loan.sort_values('Chargedoff_Proportion', ascending=False)

# Observations:
# sub Grades of "A" has very less chances of charged off.
# sub Grades of "F" and "G" have very high chances of charged off.
# proportion of charged off is increasing with sub grades moving from sub grades of "A" towards sub grades of "G"

In [ ]:
fig, ax1 = plt.subplots(figsize=(14, 8),facecolor='w')
ax1.set_title('Sub Grades vs Chargedoff Proportion',fontsize=15,color='b')
ax1=sns.barplot(x='sub_grade', y='Chargedoff_Proportion', data=grade_vs_loan, palette='coolwarm')
ax1.set_xlabel('Sub Grades',fontsize=14,color='b')
ax1.set_ylabel('Chargedoff Proportion',fontsize=14,color = 'b')
plt.show()

### Observations:
#### Sub Grades of "A" has very less chances of charged off.
#### Sub Grades of "F" and "G" have very high chances of charged off.

In [ ]:
# Employment length Vs Chargedoff_Proportion

emplen_vs_loan = loan_input.groupby(['emp_length', 'loan_status']).loan_status.count().unstack().fillna(0).reset_index()
emplen_vs_loan['Total'] = emplen_vs_loan['Charged Off'] + emplen_vs_loan['Fully Paid'] 
emplen_vs_loan['Chargedoff_Proportion'] = emplen_vs_loan['Charged Off'] / emplen_vs_loan['Total']
emplen_vs_loan.sort_values('Chargedoff_Proportion', ascending=False)

# Observations:
# Those who are not working or have less than 1 year of work experience have high chances of getting charged off.
# It makes sense as with less or no experience they don't have source of income to repay loan.
# Rest of the applicants have more or less same chnaces of getting charged off.

In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 8),facecolor='w')
ax1.set_title('Employment Length vs Chargedoff Proportion',fontsize=15,color='b')
ax1=sns.barplot(x='emp_length', y='Chargedoff_Proportion', data=emplen_vs_loan, palette='coolwarm')
ax1.set_xlabel('Employment Length',fontsize=14,color='b')
ax1.set_ylabel('Chargedoff Proportion',fontsize=14,color = 'b')
plt.show()


### Observations:
####  < 1 year of work experience have high chances of getting charged off since their income will be less to repay loan.

In [ ]:
# Interest Rate Vs Payment Term in months for every loan status.
plt.figure(figsize=(10,8), dpi=80, facecolor='w', edgecolor='k')
ax = sns.barplot(y='int_rate', x='term_in_months', data =loan_input,palette='coolwarm')
ax.set_title('Loan Payment Term vs Interest Rate',fontsize=15,color='b')
ax.set_ylabel('Interest Rate (%)',fontsize=14,color = 'b')
ax.set_xlabel('Term of loan in months',fontsize=14,color = 'b')
plt.show()

### Observations:
##### Interest rate is higher for 60 months loan term.
#### Loans issued for longer term had higher interest rates for repayement

In [ ]:
plt.figure(figsize=(10,8), dpi=80, facecolor='w', edgecolor='k')
ax = sns.barplot(y='int_rate', x='loan_amnt_cats', data =loan_input,palette='coolwarm')
ax.set_title('Loan Amount vs Interest Rate',fontsize=15,color='b')
ax.set_ylabel('Interest Rate (%)',fontsize=14,color = 'b')
ax.set_xlabel('Loan Amount Category',fontsize=14,color = 'b')
plt.show()

### Observations:
#### Interest rate is increasing with loan amount increase.

In [ ]:
plt.figure(figsize=(10,8), dpi=80, facecolor='w', edgecolor='k')
ax = sns.barplot(y='int_rate', x='dti_cats', data =loan_input,palette='coolwarm')
ax.set_title('dti vs Interest Rate',fontsize=15,color='b')
ax.set_ylabel('Interest Rate (%)',fontsize=14,color = 'b')
ax.set_xlabel('dti Category',fontsize=14,color = 'b')
plt.show()

### Observations:
#### Borrower with Low "dti" may get a lower Interest rate.
#### Slight Increase in Interest rate with increase in "dti".

In [ ]:
plt.figure(figsize=(10,8), dpi=80, facecolor='w', edgecolor='k')
ax = sns.barplot(y='int_rate', x='year', data =loan_input,palette='coolwarm')
ax.set_title('Year vs Interest Rate',fontsize=15,color='b')
ax.set_ylabel('Interest Rate (%)',fontsize=14,color = 'b')
ax.set_xlabel('Year',fontsize=14,color = 'b')
plt.show()

### Observations:
#### Year on Year Interest rate is increasing slowly.

In [ ]:
#Bivariate Analysis : Purpose Vs Interest Rate
plt.figure(figsize=(14,8),facecolor='w')
ax = sns.boxplot(x='int_rate', y='purpose', data =loan_input,palette='coolwarm')
ax.set_title('Purpose vs Interest Rate',fontsize=15,color='b')
ax.set_xlabel('Interest Rate(%)',fontsize=14,color = 'b')
ax.set_ylabel('Purpose of Loan',fontsize=14,color = 'b')
plt.show()

In [ ]:
#Bivariate Analysis : Purpose Vs Interest Rate
plt.figure(figsize=(10,8), dpi=80, facecolor='w', edgecolor='k')
ax = sns.barplot(x='int_rate', y='purpose', data =loan_input,palette='coolwarm')
ax.set_title('Purpose of Loan vs Interest Rate',fontsize=15,color='b')
ax.set_xlabel('Interest Rate (%)',fontsize=14,color = 'b')
ax.set_ylabel('Purpose',fontsize=14,color = 'b')
plt.xticks(rotation=90)
plt.show()

### Observations:
#### Interest rate is highest for small business purpose.
#### Debt consolidation falls under 2nd where borrowers had to pay more interest rate.

In [ ]:
fig, ax1 = plt.subplots(figsize=(10, 8),facecolor='w')
ax1.set_title('Interest Rate vs Chargedoff Proportion',fontsize=15,color='b')
ax1=sns.barplot(x='int_rate_cats', y='Chargedoff_Proportion', data=interest_vs_loan, palette='coolwarm')
ax1.set_xlabel('Interest Rate(%)',fontsize=14,color='b')
ax1.set_ylabel('Chargedoff Proportion',fontsize=14,color = 'b')
plt.show()

### Observations:
#### Interest rates are starting from minimin 5 %.
#### Interest rate less than 10% has very less chances of charged off
#### Interest rate more than 16% has good chnaces of charged off as compared to other category intrest rates.
#### Charged off proportion is increasing with higher interest rates

In [ ]:
# Bivariate Analysis - Grade vs Interest Rate
plt.figure(figsize=(14,8),facecolor='w')
ax = sns.barplot(y='int_rate', x='grade', data =loan_input,palette='coolwarm',order = 'ABCDEFG')
ax.set_title('Grade vs Interest Rate',fontsize=15,color='b')
ax.set_ylabel('Interest Rate',fontsize=14,color = 'b')
ax.set_xlabel('Grade',fontsize=14,color = 'b')
plt.show()

### Observations:
#### A-grade is a top grade for a lender to assign to a borrower.
#### Higher borrower's credit grade is offered with lower interest rate on a loan.
#### Interest rate is increasing with grades moving from A to F.

In [ ]:
# Bivariate Analysis - Derrogatory Public Record vs Charged off proportion
bankruptcies_vs_loan = loan_input.groupby(['pub_rec', 'loan_status']).loan_status.count().unstack().fillna(0).reset_index()
bankruptcies_vs_loan['Total']=bankruptcies_vs_loan['Charged Off']+bankruptcies_vs_loan['Fully Paid'] 
bankruptcies_vs_loan['Chargedoff_Proportion'] = bankruptcies_vs_loan['Charged Off'] / bankruptcies_vs_loan['Total']
bankruptcies_vs_loan.sort_values('Chargedoff_Proportion', ascending=False)

In [ ]:
# Bivariate Analysis - Derrogatory Public Record vs Charged off proportion
sns.set(style="whitegrid")
fig, ax1 = plt.subplots(figsize=(10, 8),facecolor='w')
ax1.set_title('Derogatory Public Record vs Chargedoff Proportion',fontsize=15,color='b')
ax1=sns.barplot(x='pub_rec', y='Chargedoff_Proportion', data=bankruptcies_vs_loan, palette='coolwarm')
ax1.set_xlabel('Derogatory Public Record',fontsize=14,color='b')
ax1.set_ylabel('Chargedoff Proportion',fontsize=14,color = 'b')
plt.show()

### Observations:
#### pub_rec value 1 or 2 have charged off chances higher than who have no Derogatory Public Record.
#### pub_rec count 3-4 has less numbers so cannot reach on any conclusions.